Setup Environment

In [1]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

Load packages

In [59]:
# Install and load additional packages needed
#install.packages("did")
library(did)
library(rio)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(hash)
.libPaths()

[1] "/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library"

DiD for BTW no controls

In [47]:
# read in the data set
btw <- import(sprintf("%s/data/btw_treat.csv", path))
# Filter for first and second vote
erst <- subset(btw, first_vote == 1)
zweit <- subset(btw, second_vote == 1)

In [18]:
colnames(btw)

[1] "AGS"           "GEN"           "year"          "Land"         
 [5] "state_id"      "Kreis"         "Wahlkreis"     "post_1998"    
 [9] "post_2002"     "post_2005"     "post_2009"     "post_2013"    
[13] "first_vote"    "second_vote"   "Wähler"        "Gültig"       
[17] "Union"         "SPD"           "FDP"           "Linke"        
[21] "Grüne"         "Andere"        "treatment_05"  "treatment_09" 
[25] "treatment_13"  "treatment_0"   "treatment_15"  "treatment_30" 
[29] "treatment_50"  "ever_treated"  "never_treated"

In [49]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = erst,
                  est_method = 'dr',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

    figures[[length(figures)+1]] <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              title = (sprintf('%s', treatments[[treatment]])),
              ncol = 3, lab_size = 10
              ) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n P-value for pre-test of parallel trends assumption: %.3f \n # Obs.: %s'
              , att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5))
    
    results[[length(results)+1]] <- result
  }
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
  final_fig <- annotate_figure(arranged_fig,
                  top = text_grob(sprintf("Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Doubly Robust', size = 10),)
  ggsave(sprintf('All_%s.png', party), plot = final_fig, path = sprintf('%s/figures/R/BTW/', path), units = 'cm', width = 21, height = 29.7, dpi="print")
  print(party)
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


In [60]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))
erst_c <- subset(btw_c, first_vote == 1)

In [63]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states_dict <- hash('SH'='Schleswig-Holstein (SH)', 'NI'='Lower Saxony (NI)', 'NW'='North-Rhine Westphalia (NRW)', 'HE'='Hesse (HE)', 
'RP'='Rhineland-Palatine (RP)', 'BW'='Baden-Württemberg (BW)', 'BY'='Bavaria (BY)', 'BB'='Brandenburg (BB)', 'MV'='Mecklenburg-Vorpommern (MV)', 
'ST'='Saxony-Anhalt (ST)', 'TH'='Thuringia (TH)') # all states - those never treated
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
for (state in states){
  for(party in parties){
    reg_data <- subset(erst_c, Land == state)
    figures  <- list()
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
      att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

      figures[[length(figures)+1]] <- ggdid(result, 
                ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
                title = (sprintf('%s', treatments_dict[[treatment]])),
                ncol = 3, lab_size = 10
                ) +
                labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n P-value for pre-test of parallel trends assumption: %.3f \n # Obs.: %s'
                , att$overall.att, att$overall.se, result$Wpval, result$n)) +
                theme(plot.caption = element_text(hjust=0.5))
      
      results[[length(results)+1]] <- result
    }
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
    final_fig <- annotate_figure(arranged_fig,
                    top = text_grob(sprintf("%s \n Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Doubly Robust', size = 10),)
    ggsave(sprintf('%s_%s.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(state)
    print(party)
  }
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "BB"
[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "BB"
[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 425 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


...

[1] "TH"
[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "TH"
[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2169 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "TH"
[1] "Andere"


DiD for BTW with controls

In [77]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))
erst_c <- subset(btw_c, first_vote == 1)

In [78]:
colnames(btw_c)

[1] "AGS"           "year"          "GEN"           "Land"         
 [5] "state_id"      "Kreis"         "Wahlkreis"     "post_1998"    
 [9] "post_2002"     "post_2005"     "post_2009"     "post_2013"    
[13] "first_vote"    "second_vote"   "Wähler"        "Gültig"       
[17] "Union"         "SPD"           "FDP"           "Linke"        
[21] "Grüne"         "Andere"        "treatment_05"  "treatment_09" 
[25] "treatment_13"  "treatment_0"   "treatment_15"  "treatment_30" 
[29] "treatment_50"  "ever_treated"  "never_treated" "pop_density"  
[33] "unemployed"    "foreign"       "catholic"      "avg_age"      
[37] "female"        "income_pc"     "east"

In [81]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ east + pop_density + unemployed + female + avg_age,
                  data = erst_c,
                  est_method = 'dr',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

    figures[[length(figures)+1]] <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              title = (sprintf('%s', treatments[[treatment]])),
              ncol = 3, lab_size = 10
              ) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n P-value for pre-test of parallel trends assumption: %.3f \n # Obs.: %s'
              , att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5))
    
    results[[length(results)+1]] <- result
  }
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
  final_fig <- annotate_figure(arranged_fig,
                  top = text_grob(sprintf("Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Doubly Robust', size = 10),)
  ggsave(sprintf('%s.png', party), plot = final_fig, path = sprintf('%s/figures/R/BTW/control', path), units = 'cm', width = 21, height = 29.7, dpi="print")
  print(party)
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


...

[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Andere"


DiD for LTW no controls

In [32]:
# read in the data set
ltw <- import(sprintf("%s/data/ltw_treat.csv", path))

In [33]:
colnames(ltw)

[1] "AGS"           "GEN"           "year"          "Land"         
 [5] "state_ID"      "Kreis"         "post_2010"     "post_2014"    
 [9] "Union"         "SPD"           "Grüne"         "FDP"          
[13] "Linke"         "Andere"        "treatment_05"  "treatment_09" 
[17] "treatment_13"  "treatment_0"   "treatment_15"  "treatment_30" 
[21] "treatment_50"  "ever_treated"  "never_treated"

In [34]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = ltw,
                  est_method = 'dr',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

    figures[[length(figures)+1]] <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              title = (sprintf('%s', treatments[[treatment]])),
              ncol = 3, lab_size = 10
              ) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n 
              P-value for pre-test of parallel trends assumption: %.3f'
              , att$overall.att, att$overall.se, result$Wpval)) +
              theme(plot.caption = element_text(hjust=0.5))
    
    results[[length(results)+1]] <- result
  }
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
  final_fig <- annotate_figure(arranged_fig,
                  top = text_grob(sprintf("Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Doubly Robust', size = 10),)
  ggsave(sprintf('%s_all.png', party), plot = final_fig, path = sprintf('%s/figures/R/LTW/', path), units = 'cm', width = 21, height = 29.7, dpi="print")
  print(party)
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1057 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1838 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 2585 units that were already treated in the first period.”


[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1057 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1838 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 2585 units that were already treated in the first period.”


[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1057 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1838 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 2585 units that were already treated in the first period.”


...

[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1057 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1838 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 2585 units that were already treated in the first period.”


[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1057 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 1838 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 2585 units that were already treated in the first period.”


[1] "Andere"


DiD for LTW no controls by state

In [36]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states <- c('NI', 'NW', 'HE', 'BY', 'TH')
keys <- keys(treatments)
results  <- list()
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw, Land == state)
    for(treatment in keys){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ 1,
                    data = reg_data,
                    est_method = 'dr',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
      att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

      figures[[length(figures)+1]] <- ggdid(result, 
                ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
                title = (sprintf('%s', treatments[[treatment]])),
                ncol = 3, lab_size = 10
                ) +
                labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n 
                P-value for pre-test of parallel trends assumption: %.3f'
                , att$overall.att, att$overall.se, result$Wpval)) +
                theme(plot.caption = element_text(hjust=0.5))
      
      results[[length(results)+1]] <- result
    }
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
    final_fig <- annotate_figure(arranged_fig,
                    top = text_grob(sprintf("Effect on %s's vote share", party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Doubly Robust', size = 10),)
    ggsave(sprintf('%s_%s.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(party)
  }
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 118 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 485 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 738 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 899 units that were already treated in the first period.”


[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 118 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 485 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 738 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 899 units that were already treated in the first period.”


[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 118 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 485 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 738 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 899 units that were already treated in the first period.”


...

[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 44 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 278 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 530 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 754 units that were already treated in the first period.”
No pre-treatment periods to test



[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 44 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 278 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 530 units that were already treated in the first period.”
No pre-treatment periods to test

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 754 units that were already treated in the first period.”
No pre-treatment periods to test



[1] "Andere"


DiD for LTW controls by state

In [82]:
ltw_c <- import(sprintf("%s/data/ltw_control.csv", path))

In [65]:
colnames(ltw_c)

[1] "AGS"           "GEN"           "year"          "Land"         
 [5] "state_ID"      "Kreis"         "post_2010"     "post_2014"    
 [9] "Union"         "SPD"           "Grüne"         "FDP"          
[13] "Linke"         "Andere"        "treatment_05"  "treatment_09" 
[17] "treatment_13"  "treatment_0"   "treatment_15"  "treatment_30" 
[21] "treatment_50"  "ever_treated"  "never_treated" "pop_density"  
[25] "unemployed"    "foreign"       "catholic"      "avg_age"      
[29] "female"        "income_pc"

In [84]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states_dict <- hash('NI'='Lower Saxony (NI)', 'NW'='North-Rhine Westphalia (NRW)', 'HE'='Hesse (HE)', 'BY'='Bavaria (BY)')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw_c, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + avg_age + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
      att <- aggte(result, type = "simple", bstrap = TRUE, clustervars = c('AGS'))

      figures[[length(figures)+1]] <- ggdid(result, 
                ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
                title = (sprintf('%s', treatments_dict[[treatment]])),
                ncol = 3, lab_size = 10
                ) +
                labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n P-value for pre-test of parallel trends assumption: %.3f \n # Obs.: %s'
                , att$overall.att, att$overall.se, result$Wpval, result$n)) +
                theme(plot.caption = element_text(hjust=0.5))
      
      results[[length(results)+1]] <- result
    }
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE, legend = 'bottom')
    final_fig <- annotate_figure(arranged_fig,
                    top = text_grob(sprintf("%s \n Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression', size = 10),)
    ggsave(sprintf('%s_%s.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state_controls', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(party)
  }
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 22 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 122 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 251 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 431 units that were already treated in the first period.”


[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 22 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 122 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 251 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 431 units that were already treated in the first period.”


[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 22 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2010.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 122 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 251 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 431 units that were already treated in the first period.”


...

[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 56 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 237 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 335 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 379 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 56 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 237 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 335 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 379 units that were already treated in the first period.”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Be aware that there are some small groups in your dataset.
  Check groups: 2014.”


[1] "Andere"


In [20]:
('ATT(SE): %.3f (%.3f) ATT(SE): %.3f (%.3f) ATT(SE): %.3f (%.3f) \n +
              P-value for pre-test of parallel trends assumption: %.3f', + 
              att$att.egt[1], att$se.egt[1], att$att.egt[2], att$att.se[2], att$att.egt[3], att$se.egt[3], result$Wpval)


Call:
aggte(MP = result, type = "group", bstrap = TRUE, clustervars = c("AGS", 
    "state_id"))

Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Journal of Econometrics, Vol. 225, No. 2, pp. 200-230, 2021. <https://doi.org/10.1016/j.jeconom.2020.12.001>, <https://arxiv.org/abs/1803.09015> 


Overall summary of ATT's based on group/cohort aggregation:  
    ATT    Std. Error     [ 95%  Conf. Int.] 
 -1.583        2.0292    -5.5601      2.3941 


Group Effects:
 Group Estimate Std. Error [95% Simult.  Conf. Band] 
  2005  -1.5059     1.7572       -5.0305      2.0188 
  2010  -2.1039     1.6440       -5.4016      1.1937 
  2014  -1.0626     3.3286       -7.7393      5.6140 
---
Signif. codes: `*' confidence band does not cover 0

Control Group:  Never Treated,  Anticipation Periods:  0
Estimation Method:  Outcome Regression